In [22]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action = 'ignore')
import os
import json
import re

In [23]:
DATA_DIR = 'C:/venvs/Programmers_DE/Final_Project/raw_data'

In [24]:
# 크롤링, API extract 완료한 데이터 로딩
web = pd.read_csv(f'{DATA_DIR}/final_recipe_website.csv')
api = pd.read_json(f'{DATA_DIR}/api.json')
with open(f'{DATA_DIR}/video.json', 'rt', encoding = 'utf-8-sig') as json_file:
    video = json.load(json_file)

In [25]:
def json_to_csv(df):
    url = []
    thumbnails = []
    title = []
    description = []
    views = []
    uploaded_date = []
    likes = []
    comments = []
    first_comment = []
    for i in range(len(video)):
        url.append(video[i]['url'])
        thumbnails.append(video[i]['thumbnail'])
        title.append(video[i]['title'])
        description.append(video[i]['description'])
        views.append(video[i]['views'])
        uploaded_date.append(video[i]['uploaded_date'])
        likes.append(video[i]['likes'])
        comments.append(video[i]['comments'])
        first_comment.append(video[i]['first_comment'])
    
    data = {
        'recipe_link': url,
        'food_img':thumbnails,
        'food_name':title,
        'ingredient':description,
        'views':views,
        'created_date':uploaded_date,
        'likes':likes,
        'comments':comments,
        'first_comments':first_comment
    }
    return pd.DataFrame(data)

In [26]:
video = json_to_csv(video)

# web

In [27]:
def website_preprocessing_ingredient(df):
    name = []
    unit = []
    for i in range(len(df)):
        name_data = df['ingredient'][1].split('ingredient')[1].split('[')[i+2].split(']')[0]
        ingredient = name_data.replace('"', '').replace("'", '')
        ingredient_name = ingredient.split(',')
        name.append(ingredient_name[:10])
        
        unit_data = df['ingredient'][1].split('ingredient')[2].split('[')[i+2].split(']')[0]
        ingre_unit = unit_data.replace('"', '').replace("'", '')
        ingre_unit = ingre_unit.split(',')
        unit.append(ingre_unit[:10])
    
    for i in range(len(name)):
        while len(name[i]) < 10:
            name[i].append(np.nan)
        
        while len(unit[i]) < 10:
            unit[i].append(np.nan)
    
    name_df = pd.DataFrame(name, columns = ['재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10'])
    unit_df = pd.DataFrame(unit, columns = ['재료1_정량', '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량', '재료8_정량', '재료9_정량', '재료10_정량'])
    merged_df = pd.merge(df, name_df, left_index=True, right_index=True)
    merged = pd.merge(merged_df, unit_df, left_index=True, right_index=True)
    return merged

In [28]:
web_df = website_preprocessing_ingredient(web)

# API

In [29]:
def api_preprocessing_metadata(df):
    ingredient_set = set()
    name_dt = []
    for i in range(len(df)):
        data = df['ingredient'][i]
        if data is None:
            data = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ,np.nan, np.nan, np.nan, np.nan]
            name_dt.append(data)
        else:
            data = data.replace(' ', '').replace('조림장 :', '').replace('고명 :', '').replace('(채)', '').replace('\n', '').replace('●', '').replace('주재료 : ', '').replace('재료 ', '').replace('양념장 :', '').split(',')
            ch = []
            u = []
            for i in data:
                k = i.split('(')[0]
                ch.append(k)
            for i in range(len(ch)):
                if len(ch) > 10:
                    ch = ch[:10]
                else:
                    while len(ch) < 10:
                        ch.append(np.nan)
            
            name_dt.append(ch)
    
    data_pre = []
    data_unit = []
    for i in range(len(name_dt)):
        if name_dt[i] == [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan ,np.nan, np.nan, np.nan, np.nan]:
            data_pre.append(name_dt[i])
            data_unit.append(name_dt[i])
            continue
        else:
            ingre_name = []
            ingre_unit = []
            for j in name_dt[i]:
                if isinstance(j, str):
                    match = re.match(r'^(.*?)([0-9/]+[^\d]*)$', j)
                    if match:
                        name = match.group(1).strip()
                        amount = match.group(2).strip()
                        ingre_name.append(name)
                        ingre_unit.append(amount)
            data_pre.append(ingre_name)
            data_unit.append(ingre_unit)  

    for i in range(len(data_pre)):
        if len(data_pre[i]) > 10:
            data_pre[i] = data_pre[i][:10]
            data_unit[i] = data_unit[i][:10]
        else:
            while len(data_pre[i]) < 10:
                data_pre[i].append(np.nan)
                data_unit[i].append(np.nan)
                
    name_df = pd.DataFrame(data_pre,  columns = ['재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10'])
    unit_df = pd.DataFrame(data_unit, columns = ['재료1_정량', '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량', '재료8_정량', '재료9_정량', '재료10_정량'])
    merged_df = pd.merge(df, name_df, left_index=True, right_index=True)
    merged = pd.merge(merged_df, unit_df, left_index=True, right_index=True)
    
    merged['recipe_link'] = merged['recipe_01'] + '\n' + merged['recipe_02'] + '\n' + merged['recipe_03'] + '\n' + merged['recipe_04'] + '\n' + merged['recipe_05'] + '\n' + merged['recipe_06']
    return merged

In [30]:
api_df = api_preprocessing_metadata(api)

# video

In [31]:
def video_ingredient_separate(df):
    ingre = []
    for i in range(len(df)):
        if '[ 재료 ]' in df['ingredient'][i] and '[ 만드는 법 ]' in df['ingredient'][i]:
            food_name = df['ingredient'][i].split('[ 재료 ]')[1].split('[ 만드는 법 ]')[0]
            ingre.append(food_name)
        elif '[ 재료 ]' in df['ingredient'][i] and '[만드는 법]' in df['ingredient'][i]:
            food_name = df['ingredient'][i].split('[ 재료 ]')[1].split('[만드는 법]')[0]
            ingre.append(food_name)
        elif '[재료]' in df['ingredient'][i] and '\n\n[볶아서 만드는 법]\n\n' in df['ingredient'][i]:
            food_name = df['ingredient'][i].split('[재료]')[1].split('\n\n[볶아서 만드는 법]\n\n')[0]
            ingre.append(food_name)    
        elif '[재료]' in df['ingredient'][i] and '[만드는 법]' in df['ingredient'][i]:
            food_name = df['ingredient'][i].split('[재료]')[1].split('[만드는 법]')[0]
            ingre.append(food_name)
        elif '[재료]' in df['ingredient'][i] and '[조리법]' in df['ingredient'][i]:
            food_name = df['ingredient'][i].split('[재료]')[1].split('[조리법]')[0]
            ingre.append(food_name)
        else:
            ingre.append(np.nan)
    fin = []
    for i in ingre:
        ksd = []
        if i is np.nan:
            i = [np.nan] * 10
            fin.append(i)
        else:
            k = i.split('\n')
            fin.append(k)   
    detail_fin = []
    for li in fin:
        if li == [np.nan] * 10:
            detail_fin.append(li)
            continue
        else:
            dk = []
            for i in li:
                if i == '' or i == ' ':
                    continue
                else:
                    dk.append(i)
            detail_fin.append(dk)
    last_pre = []
    for li in detail_fin:
        if li == [np.nan] * 10:
            last_pre.append(li)
        elif len(li) < 10:
            while len(li) < 10:
                li.append(np.nan)
            last_pre.append(li)
        elif len(li) > 10:
            li = li[:10]
            last_pre.append(li)
        elif len(li) == 10:
            last_pre.append(li)  
    ch = []
    for food_ingre in last_pre:
        detail = []
        if food_ingre == [np.nan] * 10:
            ch.append(food_ingre)
        else:
            for i in food_ingre:
                if isinstance(i, str) and '(' in i:
                    k = i.split('(')[0]
                    detail.append(k)
                else:
                    detail.append(i)
            ch.append(detail)     
    parsed_data = []
    parsed_amount = []
    for recipe in ch:
        parsed_ingre = []
        parsed_unit = []
        if recipe == [np.nan] * 10:
            parsed_data.append(recipe)
            parsed_amount.append(recipe)
            continue
        else:
            for ingredient in recipe:
                if isinstance(ingredient, str):
                    match = re.match(r'^(.*?)([0-9/]+[^\d]*)$', ingredient)
                    if match:
                        name = match.group(1).strip()
                        amount = match.group(2).strip()
                        parsed_ingre.append(name)
                        parsed_unit.append(amount)
                    
        parsed_data.append(parsed_ingre)
        parsed_amount.append(parsed_unit)
                               
    ingredient_df = pd.DataFrame(parsed_data, columns = ['재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10'])
    unit_df = pd.DataFrame(parsed_amount, columns = ['재료1_정량', '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량', '재료8_정량', '재료9_정량', '재료10_정량'])
    merged_df = pd.merge(df, ingredient_df, left_index=True, right_index=True)
    merged = pd.merge(merged_df, unit_df, left_index=True, right_index=True)
    return merged

In [32]:
video_df = video_ingredient_separate(video)

# Data Merge

In [33]:
web_df['category_high_level'] = np.nan
web_df['category_mid_level'] = np.nan
web_df['category_low_level'] = np.nan
web_df['likes'] = 0
web_df['ID'] = range(5000000, 5000000 + len(web))
web_df['data_source'] = 'website'

api_df['food_img'] = api_df['food_img_1']
api_df['views'] = 0
api_df['likes'] = 0
api_df['comments'] = 0
api_df['reviews'] = 0
api_df['category_high_level'] = np.nan
api_df['category_mid_level'] = api_df['category_2']
api_df['category_low_level'] = api_df['category']
api_df['created_date'] = '2022-08-12'
api_df['ID'] = api_df['API_ID']
api_df['data_source'] = 'API'

video_df['category_high_level'] = np.nan
video_df['category_mid_level'] = np.nan
video_df['category_low_level'] = np.nan
video_df['ID'] = range(2000000, 2000000 + len(video))
video_df['serving'] = np.nan
video_df['reviews'] = 0
video_df['data_source'] = 'youtube'

In [34]:
web_df_pre = web_df[['data_source', 'ID', 'food_name', 'food_img', 'recipe_link', 'serving', 'category_high_level', 'category_mid_level', 'category_low_level',
                    'created_date', 'views', 'likes', 'reviews', 'comments',  '재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10',
                    '재료1_정량', '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량','재료8_정량', '재료9_정량', '재료10_정량']]
api_df_pre = api_df[['data_source', 'ID', 'food_name', 'food_img', 'recipe_link', 'serving', 'category_high_level', 'category_mid_level', 'category_low_level',
                    'created_date', 'views', 'likes', 'reviews', 'comments',  '재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10',
                    '재료1_정량', '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량','재료8_정량', '재료9_정량','재료10_정량']]
video_df_pre = video_df[['data_source', 'ID', 'food_name', 'food_img', 'recipe_link', 'serving', 'category_high_level', 'category_mid_level', 'category_low_level',
                    'created_date', 'views', 'likes', 'reviews', 'comments',  '재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10',
                    '재료1_정량', '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량','재료8_정량', '재료9_정량','재료10_정량']]

In [37]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def rating_dataframe(df1, df2, df3):
    for i in range(len(df3)):
        if df3.loc[i, 'comments'] == '':
            df3.loc[i, 'comments'] = np.nan
    
    df3['comments'] = df3['comments'].fillna(0)
    
    df1['views'] = df1['views'].astype('int64')
    df1['likes'] = df1['likes'].astype('int64')
    df1['comments'] = df1['comments'].astype('int64')
    df1['reviews'] = df1['reviews'].astype('int64')
    
    df2['views'] = df2['views'].astype('int64')
    df2['likes'] = df2['likes'].astype('int64')
    df2['comments'] = df2['comments'].astype('int64')
    df2['reviews'] = df2['reviews'].astype('int64')
    
    df3['views'] = df3['views'].astype('int64')
    df3['likes'] = df3['likes'].astype('int64')
    df3['comments'] = df3['comments'].astype('int64')
    
    df1['rating'] = df1['views'] + df1['likes'] + df1['comments'] + df1['reviews']
    df2['rating'] = df2['views'] + df2['likes'] + df2['comments'] + df2['reviews']
    df3['rating'] = df3['views'] + df3['likes'] + df3['comments'] + df3['reviews']
    
    scaler = StandardScaler()

    # Normalize the 'rating' column in each dataframe
    df1['rating'] = scaler.fit_transform(df1[['rating']])
    df2['rating'] = scaler.fit_transform(df2[['rating']])
    df3['rating'] = scaler.fit_transform(df3[['rating']])
    
    df = pd.concat([df1, df2, df3])
    return df

In [38]:
df_pre = rating_dataframe(web_df_pre, api_df_pre, video_df_pre)

In [42]:
df_pre.describe()

,ID,category_high_level,views,likes,reviews,comments,rating
count,1.446000e+03,0.0,1.446000e+03,1446.000000,1446.0,1446.000000,1.446000e+03
mean,4.808492e+05,NaN,4.866756e+05,5723.334716,0.0,230.486169,1.458799e-17
std,9.127590e+05,NaN,1.393101e+06,15903.743559,0.0,695.435419,4.793306e-01
min,1.800000e+01,NaN,0.000000e+00,0.000000,0.0,0.000000,-1.002513e+00
25%,5.212500e+02,NaN,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00
50%,9.455000e+02,NaN,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00
75%,3.273750e+03,NaN,0.000000e+00,0.000000,0.0,0.000000,0.000000e+00
max,5.000009e+06,NaN,1.293144e+07,147550.000000,0.0,10758.000000,4.807973e+00
